In [2]:
!pip install requests
!pip install python-docx

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB 640.0 kB/s eta 0:00:06
    --------------------------------------- 0.1/3.8 MB 660.6 kB/s eta 0:00:06
   - -------------------------------------- 0.2/3.8 MB 1.6 MB/s eta 0:00:03
   ---- ----------------------------------- 0.4/3.8 MB 2.7 MB/s eta 0:00:02
   --------- ------------------------------ 0.9/3.8 MB 4.2 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 5.3 MB/s eta 0:00:01
   ------------------- -------------------- 1.9/3.8 MB 6.3 MB/s eta 0:00:01
   ---------------------------- ----------- 2.7/3.8 MB 7.8 MB/s eta 0:00:01
   ---------------------------------- ----- 3.3/3.8 MB 8.5 MB/s eta 0:00:01
   ----------------------------------------


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import json
import base64
import os
import re
from docx import Document
from docx.shared import Inches



#Get_accesscode 用于向百度获取access token key，第一个参数user_api是创建的应用API，第二个参数user_key是创建应用的key

In [3]:
def get_accesscode(user_api:str,user_key:str)->str:
    url = "https://aip.baidubce.com/oauth/2.0/token?client_id="+user_api+"&client_secret="+user_key+"&grant_type=client_credentials"
    
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    access_token = response.json()['access_token']
    return access_token

In [4]:
access_token_link = get_accesscode("rR0nKpwIcY3raJhVXcAdMD2n","KlL7eHPpBOr6xH8nj3ve0qHJcMk27Slf")

In [5]:
print(access_token_link)

24.09bb9cde27d7caf80f20f18478848850.2592000.1725592423.282335-100297428


写入word文档


In [7]:
from docx import Document
from docx.shared import Inches

def write_into_new_word(heading:str,name:str,response):
    # 创建一个 Word 文档对象
    document = Document()
    # 添加标题
    document.add_heading(heading, 0)
    #添加段落
    document.add_paragraph("Subject: Chinese")
    document.add_paragraph("Level: Secondary 2 / S2 / F2")
    document.add_paragraph("Region: Hong Kong")
    document.add_paragraph("")
    #document.add_paragraph("Note: (a^2) mean the power of a is 2")
    #下面的for loop是讲扫描到的写入word文档，用不同的OCR写法不一样，可以先print一下
    #response.json去看看格式然后去改下面的格式
    for i in range(len(response.json()['words_result'])):
        document.add_paragraph(response.json()['words_result'][i]['words'])
    '''
    for i in range(len(response.json()['results'])):
        document.add_paragraph(response.json()['results'][i]['words']['word']) 
        #text = text + response.json()['results'][i]['words']['word'] +' '
    '''
    document.save(name)

往现有的word文档里面填写资料

In [8]:

def write_into_exist_word(name:str,response)->None:
    # 打开现有的 Word 文档
    document = Document(name)

    #document.add_paragraph("Note: (a^2) mean the power of a is 2")
    #下面的for loop是讲扫描到的写入word文档，用不同的OCR写法不一样，可以先print一下
    #response.json去看看格式然后去改下面的格式
    for i in range(len(response.json()['words_result'])):
        document.add_paragraph(response.json()['words_result'][i]['words'])
    '''
    for i in range(len(response.json()['results'])):
        document.add_paragraph(response.json()['results'][i]['words']['word'])
        #text = text + response.json()['results'][i]['words']['word'] +' '
    '''
    # 保存文档
    document.save(name)

查看文件路径是否按顺序

In [10]:
import os
import re

folder_path = 'Engwriting'

# 提取文件名中的数字部分并进行排序
def sort_key(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else float('inf')

# 获取文件列表并按数字排序
filenames = sorted(os.listdir(folder_path), key=sort_key)

# 打印排序后的文件名
for filename in filenames:
    print(filename)

img5.jpg
img11.jpg
img15.jpg
img19.jpg
img23.jpg
img27.jpg


In [11]:
import os
#改成你个人的图片集
folder_path = 'Engwriting'
# 二进制方式打开图片文件
request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate_basic"
First_control = True

for filename in sorted(os.listdir(folder_path), key=sort_key):
    f = open(folder_path+"/"+filename, 'rb')
    img = base64.b64encode(f.read())
    params = {"image":img,"language_type":"CHN_ENG","result_type":"big"}
    access_token = access_token_link
    request_url = request_url + "?access_token=" + access_token
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    response = requests.post(request_url, data=params, headers=headers)
    if response:
        print (response.json())
    if First_control:
        write_into_new_word('Engwriting','writing.docx',response)
        
        First_control = False
    else:
        write_into_exist_word('writing.docx',response)

{'words_result': [{'words': "St. Edward's Catholic Primary School"}, {'words': 'Primary 6 English'}, {'words': '2023-2024 Unit Assessment 3'}, {'words': 'Answer Sheet'}, {'words': 'Name:'}, {'words': 'Lucas lio'}, {'words': '11'}, {'words': ')'}, {'words': 'Class: 6'}, {'words': 'A'}, {'words': 'Write an e-mail to your friend Carrie about what happened. Write at least 100 words.'}, {'words': 'Hi Carrie,'}, {'words': 'How are you? Something happened to me yesterday.'}, {'words': 'On that day,I'}, {'words': 'Llas late to wake up so I as fast as a'}, {'words': 'roket to brush my teeth and dreed my clothes'}, {'words': 'then I went out.'}, {'words': "Next,I went my school by my dod's cau."}, {'words': 'Suddenly,the car was stop so my pencil case'}, {'words': 'fall out but I was talk to my dad so I dik'}, {'words': 'h not know the pencil case fall out.'}, {'words': 'Then,I went in to my school.Tonday was.'}, {'words': "my test,but I found I haven't boring my"}, {'words': 'pencil case Oh no!